## Training SageMaker Models using the DGL with Pytorch backend
The **SageMaker Python SDK** makes it easy to train DGL models. In this example, we train a simple graph neural network using the [DMLC DGL API](https://github.com/dmlc/dgl.git) and the [Cora dataset](https://relational.fit.cvut.cz/dataset/CORA). The Cora dataset describes a citation network. The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. The task at hand is to train a node classification model using Cora dataset. 

### Setup
We need to define a few variables that will be needed later in the example.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sess.default_bucket()

# Location to put your custom code.
custom_code_upload_location = 'customcode'

# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

### The training script
The pytorch_gcn.py script provides all the code we need for training a SageMaker model. 

In [ ]:
!cat pytorch_gcn.py

### Get DGL docker image (Optional)
We provide dgl-0.4 gpu-docker at dockerhub under dgllib registry. You can pull it yourself and push it into your AWS ECR. Following script helps you to do so. You can skip this step, if you have already got/prepared your dgl docker image in you ECR.

In [ ]:
%%sh
default_docker_name="dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0"
docker pull $default_docker_name

docker_name=sagemaker-dgl-pytorch-gcn

account=$(aws sts get-caller-identity --query Account --output text)
echo $account
region=$(aws configure get region)
region=${region:-us-east-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${docker_name}:latest"
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${docker_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${docker_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

docker tag ${default_docker_name} ${fullname}

docker push ${fullname}

### SageMaker's  estimator class
The SageMaker Estimator allows us to run single machine in SageMaker, using CPU or GPU-based instances.

When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role. We also provide a few other parameters. train_instance_count and train_instance_type determine the number and type of SageMaker instances that will be used for the training job. The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the pytorch_gcn.py script above.

The entrypoint of sagemaker docker (e.g., dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0) is a **train** script under /usr/bin/. The **train** script inside dgl docker image provided in the above will try to get the **real entrypoint** from hyperparameters and run the **real entrypoint** under **'training-code' data channel** (/opt/ml/input/data/training-code/) .

For this example, we will choose one ml.p3.2xlarge instance

In [ ]:
import boto3

# Set target dgl-docker name
docker_name='sagemaker-dgl-pytorch-gcn'

CODE_PATH = 'pytorch_gcn.py'
code_location = sess.upload_data(CODE_PATH, bucket=bucket, key_prefix=custom_code_upload_location)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, docker_name)
print(image)

params = {}
params['entrypoint'] = CODE_PATH
params['dataset'] = 'cora'
params['gpu'] = 0
estimator = sagemaker.estimator.Estimator(image,
                        role, 
                        train_instance_count=1, 
                        train_instance_type='ml.p3.2xlarge',
                        hyperparameters=params,
                        sagemaker_session=sess)

### Running the Training Job
After we've constructed our Estimator object, we can fit it using sagemaker (The dataset will be automatically downloaded). Below we run SageMaker training on one channels: training-code, the code to run.

In [ ]:
data = {}
data['training-code'] = code_location

estimator.fit(data)